# S&P 500 Value Screener — Top 20 Stocks (5-Year Horizon)

This notebook screens the entire S&P 500 and ranks stocks using a three-pillar composite score:

| Pillar | Weight | What it measures |
|---|---|---|
| **DCF Margin of Safety** | 40% | Intrinsic value vs. current price (via discounted FCF) |
| **Fundamental Ratios** | 35% | P/E, P/FCF, EV/EBITDA, P/B, PEG — sector-adjusted |
| **Quality Score** | 25% | ROE, ROIC, debt levels, FCF consistency |

**Designed for a 5-year investment horizon.**

> **Runtime note:** Fetching data for ~500 stocks takes 10–20 minutes due to Yahoo Finance rate limits. Results are cached to `data/raw_data.pkl` so you can re-run analysis cells instantly after the first run.

## 0. Imports & Configuration

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import time
import pickle
import requests
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

# ── Plot style ──────────────────────────────────────────────────────────────
plt.rcParams.update({
    'figure.facecolor': '#0f1117',
    'axes.facecolor':   '#0f1117',
    'axes.edgecolor':   '#333',
    'axes.labelcolor':  '#ccc',
    'xtick.color':      '#aaa',
    'ytick.color':      '#aaa',
    'text.color':       '#eee',
    'grid.color':       '#222',
    'grid.linestyle':   '--',
    'font.family':      'DejaVu Sans',
    'figure.dpi':       120,
})
sns.set_style('dark')

# ── DCF Parameters ──────────────────────────────────────────────────────────
WACC              = 0.10   # Discount rate (10% — reasonable all-in cost of capital)
TERMINAL_GROWTH   = 0.03   # Long-run perpetual growth (≈ GDP growth)
PROJECTION_YEARS  = 5      # Match the 5-year investment horizon
MAX_FCF_GROWTH    = 0.25   # Cap bullish growth assumptions at 25% p.a.
MIN_FCF_GROWTH    = -0.05  # Floor at -5% for declining businesses
MARGIN_OF_SAFETY  = 0.20   # Only "attractive" if trading ≥20% below DCF value

# ── Quality thresholds (minimum to pass quality gate) ───────────────────────
MIN_ROE           = 0.05   # 5%  (relaxed to avoid excluding cyclicals unfairly)
MIN_ROIC          = 0.04   # 4%
MAX_DEBT_EQUITY   = 5.0    # Exclude extreme leverage

# ── Scoring weights ──────────────────────────────────────────────────────────
W_DCF             = 0.40
W_FUNDAMENTALS    = 0.35
W_QUALITY         = 0.25

os.makedirs('data', exist_ok=True)
os.makedirs('output', exist_ok=True)
print('✓ Setup complete')

## 1. Fetch S&P 500 Tickers

In [ ]:
def get_sp500_tickers() -> pd.DataFrame:
    """Scrape S&P 500 constituents from Wikipedia."""
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    resp = requests.get(url, timeout=15)
    soup = BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'id': 'constituents'})
    df = pd.read_html(str(table))[0]
    df.columns = df.columns.str.strip()
    df = df[['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry']].copy()
    df.columns = ['ticker', 'name', 'sector', 'industry']
    # Yahoo Finance uses '-' instead of '.' for some tickers (e.g. BRK.B → BRK-B)
    df['ticker'] = df['ticker'].str.replace('.', '-', regex=False)
    return df

sp500 = get_sp500_tickers()
print(f'Loaded {len(sp500)} S&P 500 companies')
print(f'Sectors: {sp500["sector"].nunique()}')
sp500.head(5)

## 2. Fetch Fundamental Data from Yahoo Finance

> **This cell takes ~10–20 minutes on first run.** Results are cached — skip to Cell 3 on subsequent runs.

In [ ]:
CACHE_FILE = 'data/raw_data.pkl'
FORCE_REFRESH = False   # Set True to re-download everything

def safe_get(d, *keys, default=np.nan):
    """Safely traverse nested dict/Series."""
    for k in keys:
        try:
            if isinstance(d, dict):
                d = d.get(k, default)
            else:
                d = d[k]
        except (KeyError, TypeError, IndexError):
            return default
    return d if d is not None else default

def extract_ttm_fcf(cashflow_df: pd.DataFrame) -> float:
    """Extract trailing twelve-month free cash flow."""
    if cashflow_df is None or cashflow_df.empty:
        return np.nan
    try:
        # yfinance annual cashflow: rows = line items, cols = years
        ocf_row = None
        capex_row = None
        for label in ['Operating Cash Flow', 'Total Cash From Operating Activities']:
            if label in cashflow_df.index:
                ocf_row = cashflow_df.loc[label]
                break
        for label in ['Capital Expenditure', 'Capital Expenditures']:
            if label in cashflow_df.index:
                capex_row = cashflow_df.loc[label]
                break
        if ocf_row is None:
            return np.nan
        ocf   = float(ocf_row.iloc[0])
        capex = float(capex_row.iloc[0]) if capex_row is not None else 0.0
        # capex is usually negative in yfinance; FCF = OCF + capex
        return ocf + capex
    except Exception:
        return np.nan

def fcf_growth_rate(cashflow_df: pd.DataFrame) -> float:
    """Compute 3-year FCF CAGR as growth estimate for DCF."""
    if cashflow_df is None or cashflow_df.empty:
        return np.nan
    try:
        ocf_row = None
        capex_row = None
        for label in ['Operating Cash Flow', 'Total Cash From Operating Activities']:
            if label in cashflow_df.index:
                ocf_row = cashflow_df.loc[label]
                break
        for label in ['Capital Expenditure', 'Capital Expenditures']:
            if label in cashflow_df.index:
                capex_row = cashflow_df.loc[label]
                break
        if ocf_row is None or len(ocf_row) < 2:
            return np.nan
        ocf   = ocf_row.values.astype(float)
        capex = capex_row.values.astype(float) if capex_row is not None else np.zeros(len(ocf))
        fcf   = ocf + capex
        n = min(len(fcf), 4)   # up to 4 years of history
        start, end = fcf[n-1], fcf[0]
        if start <= 0 or end <= 0:
            return np.nan
        return (end / start) ** (1 / (n - 1)) - 1
    except Exception:
        return np.nan

def fetch_ticker_data(ticker: str) -> dict:
    """Fetch all needed data for a single ticker."""
    try:
        t = yf.Ticker(ticker)
        info = t.info or {}
        cf   = t.cashflow          # annual cash-flow statement
        bs   = t.balance_sheet     # annual balance sheet

        price          = safe_get(info, 'currentPrice') or safe_get(info, 'regularMarketPrice')
        market_cap     = safe_get(info, 'marketCap')
        shares         = safe_get(info, 'sharesOutstanding')
        pe             = safe_get(info, 'trailingPE')
        pb             = safe_get(info, 'priceToBook')
        peg            = safe_get(info, 'pegRatio')
        ev_ebitda      = safe_get(info, 'enterpriseToEbitda')
        roe            = safe_get(info, 'returnOnEquity')
        roa            = safe_get(info, 'returnOnAssets')
        debt_equity    = safe_get(info, 'debtToEquity')
        total_debt     = safe_get(info, 'totalDebt')
        cash           = safe_get(info, 'totalCash')
        ev             = safe_get(info, 'enterpriseValue')
        ebitda         = safe_get(info, 'ebitda')
        revenue        = safe_get(info, 'totalRevenue')
        eps_fwd        = safe_get(info, 'forwardEps')
        revenue_growth = safe_get(info, 'revenueGrowth')
        earnings_growth= safe_get(info, 'earningsGrowth')
        gross_margins  = safe_get(info, 'grossMargins')
        op_margins     = safe_get(info, 'operatingMargins')
        profit_margins = safe_get(info, 'profitMargins')

        # FCF from cash flow statement
        ttm_fcf   = extract_ttm_fcf(cf)
        fcf_growh = fcf_growth_rate(cf)

        # P/FCF
        p_fcf = np.nan
        if market_cap and ttm_fcf and ttm_fcf > 0:
            p_fcf = market_cap / ttm_fcf

        # ROIC proxy: EBIT*(1-tax) / (total_debt + equity)
        roic = np.nan
        try:
            ebit = safe_get(info, 'ebit')
            tax_rate = safe_get(info, 'effectiveTaxRate', default=0.21)
            equity = safe_get(info, 'bookValue')
            if shares and equity:
                equity_total = equity * shares
            else:
                equity_total = np.nan
            if ebit and equity_total and not np.isnan(equity_total):
                invested_cap = (total_debt or 0) + equity_total
                if invested_cap > 0:
                    roic = ebit * (1 - (tax_rate or 0.21)) / invested_cap
        except Exception:
            pass

        return {
            'ticker':          ticker,
            'price':           price,
            'market_cap':      market_cap,
            'shares':          shares,
            'pe':              pe,
            'pb':              pb,
            'peg':             peg,
            'ev_ebitda':       ev_ebitda,
            'p_fcf':           p_fcf,
            'roe':             roe,
            'roa':             roa,
            'roic':            roic,
            'debt_equity':     debt_equity,
            'total_debt':      total_debt,
            'cash':            cash,
            'ev':              ev,
            'ebitda':          ebitda,
            'revenue':         revenue,
            'ttm_fcf':         ttm_fcf,
            'fcf_growth':      fcf_growh,
            'revenue_growth':  revenue_growth,
            'earnings_growth': earnings_growth,
            'gross_margins':   gross_margins,
            'op_margins':      op_margins,
            'profit_margins':  profit_margins,
            'eps_fwd':         eps_fwd,
        }
    except Exception as e:
        return {'ticker': ticker, 'error': str(e)}

# ── Main fetch loop ──────────────────────────────────────────────────────────
if os.path.exists(CACHE_FILE) and not FORCE_REFRESH:
    with open(CACHE_FILE, 'rb') as f:
        raw_records = pickle.load(f)
    print(f'✓ Loaded cached data ({len(raw_records)} records). Set FORCE_REFRESH=True to re-download.')
else:
    print(f'Fetching data for {len(sp500)} tickers from Yahoo Finance...')
    print('  This will take ~15 minutes. Progress is saved — restart is safe.\n')
    raw_records = []
    tickers = sp500['ticker'].tolist()

    for i, ticker in enumerate(tqdm(tickers, desc='Fetching')):
        rec = fetch_ticker_data(ticker)
        raw_records.append(rec)
        # Polite delay to avoid rate-limiting (1 req/sec on average)
        time.sleep(0.8)
        # Checkpoint every 50 tickers
        if (i + 1) % 50 == 0:
            with open(CACHE_FILE, 'wb') as f:
                pickle.dump(raw_records, f)

    with open(CACHE_FILE, 'wb') as f:
        pickle.dump(raw_records, f)
    print(f'\n✓ Done. Data cached to {CACHE_FILE}')

## 3. Build the Master DataFrame

In [ ]:
df_raw = pd.DataFrame(raw_records)

# Merge with sector/name metadata
df = df_raw.merge(sp500, on='ticker', how='left')

# Drop tickers with critical missing data
df = df[df['price'].notna() & df['market_cap'].notna()]
df = df[~df.get('error', pd.Series(dtype=str)).notna() if 'error' in df.columns else df.index]

# Ensure numeric types
numeric_cols = ['pe', 'pb', 'peg', 'ev_ebitda', 'p_fcf', 'roe', 'roa',
                'roic', 'debt_equity', 'ttm_fcf', 'fcf_growth',
                'revenue_growth', 'earnings_growth', 'gross_margins',
                'op_margins', 'profit_margins', 'price', 'market_cap']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# debt_equity from yfinance is sometimes in % form (e.g. 45.3 = 45.3%)
# normalise to ratio form if values are very large
if df['debt_equity'].median() > 10:
    df['debt_equity'] = df['debt_equity'] / 100

print(f'Universe after cleaning: {len(df)} stocks')
print(f'\nData coverage (non-null %):')
coverage = df[numeric_cols].notna().mean().sort_values(ascending=False)
for col, pct in coverage.items():
    bar = '█' * int(pct * 20)
    print(f'  {col:<18} {bar:<20} {pct:.0%}')

## 4. DCF Valuation

**Model:**
1. Start with trailing FCF per share
2. Grow at the company's historical FCF CAGR (capped at ±25%/year) for 5 years
3. Apply a terminal value using the Gordon Growth Model (3% perpetual growth)
4. Discount all cash flows at 10% WACC
5. Add net cash (cash − debt) per share
6. **Margin of safety** = (intrinsic value − price) / intrinsic value

In [ ]:
def run_dcf(row: pd.Series) -> dict:
    """Return intrinsic value per share and margin of safety for one stock."""
    result = {'dcf_value': np.nan, 'margin_of_safety': np.nan}

    fcf        = row.get('ttm_fcf')
    shares     = row.get('shares')
    price      = row.get('price')
    fcf_g      = row.get('fcf_growth')
    cash       = row.get('cash') or 0
    total_debt = row.get('total_debt') or 0

    # Need positive FCF, share count, and price
    if any(pd.isna(x) for x in [fcf, shares, price]):
        return result
    if shares <= 0 or price <= 0 or fcf <= 0:
        return result

    # Growth rate: use historical FCF CAGR, fall back to 5% if unavailable
    if pd.isna(fcf_g):
        fcf_g = 0.05
    g = np.clip(fcf_g, MIN_FCF_GROWTH, MAX_FCF_GROWTH)

    # Project FCF for PROJECTION_YEARS
    pv_sum = 0.0
    for yr in range(1, PROJECTION_YEARS + 1):
        projected_fcf = fcf * (1 + g) ** yr
        pv_sum += projected_fcf / (1 + WACC) ** yr

    # Terminal value (Gordon Growth at end of projection period)
    terminal_fcf = fcf * (1 + g) ** PROJECTION_YEARS * (1 + TERMINAL_GROWTH)
    terminal_val = terminal_fcf / (WACC - TERMINAL_GROWTH)
    pv_terminal  = terminal_val / (1 + WACC) ** PROJECTION_YEARS

    # Enterprise value → equity value
    equity_value = pv_sum + pv_terminal + cash - total_debt
    if equity_value <= 0:
        return result

    intrinsic_value = equity_value / shares
    mos = (intrinsic_value - price) / intrinsic_value

    result['dcf_value']         = round(intrinsic_value, 2)
    result['margin_of_safety']  = round(mos, 4)
    return result

dcf_results = df.apply(run_dcf, axis=1, result_type='expand')
df = pd.concat([df, dcf_results], axis=1)

dcf_valid = df['margin_of_safety'].notna().sum()
undervalued = (df['margin_of_safety'] >= MARGIN_OF_SAFETY).sum()
print(f'DCF computed for {dcf_valid} stocks')
print(f'Stocks trading ≥{MARGIN_OF_SAFETY:.0%} below DCF value: {undervalued}')

# Sanity-check sample
df[['ticker','name','price','dcf_value','margin_of_safety']].dropna().sort_values(
    'margin_of_safety', ascending=False).head(10)

## 5. Composite Scoring

Each metric is percentile-ranked **within its GICS sector** (so Financials are compared to Financials, Tech to Tech, etc.) to avoid penalising sectors that structurally carry different valuations.

For each metric, the ranking direction is set so that **higher score = more attractive**.

In [ ]:
def percentile_rank_sector(series: pd.Series, sector: pd.Series,
                            ascending: bool = True) -> pd.Series:
    """
    Rank each value within its sector group.
    ascending=True  → lower value is better (e.g. P/E)
    ascending=False → higher value is better (e.g. ROE)
    Returns a 0-1 percentile score where 1.0 = most attractive.
    """
    result = pd.Series(np.nan, index=series.index)
    for sec in sector.unique():
        mask = sector == sec
        sub  = series[mask]
        valid = sub.notna()
        if valid.sum() < 3:
            continue
        ranked = sub[valid].rank(pct=True, ascending=ascending)
        # ascending=True means low value → high rank (good), which is what we want
        # But rank(ascending=True) gives LOW values a LOW rank.
        # Flip: if lower is better, we want low values to score 1.0
        if ascending:  # lower metric = better
            result[valid & mask] = 1 - ranked + ranked.min()
        else:          # higher metric = better
            result[valid & mask] = ranked
    # Normalise to [0, 1]
    mn, mx = result.min(), result.max()
    if mx > mn:
        result = (result - mn) / (mx - mn)
    return result

sector = df['sector'].fillna('Unknown')

# ── Valuation sub-scores (lower = better → ascending=True) ──────────────────
df['score_pe']       = percentile_rank_sector(df['pe'].clip(0, 100),      sector, ascending=True)
df['score_pfcf']     = percentile_rank_sector(df['p_fcf'].clip(0, 150),   sector, ascending=True)
df['score_ev_ebitda']= percentile_rank_sector(df['ev_ebitda'].clip(0, 60),sector, ascending=True)
df['score_pb']       = percentile_rank_sector(df['pb'].clip(0, 20),       sector, ascending=True)
df['score_peg']      = percentile_rank_sector(df['peg'].clip(0, 5),       sector, ascending=True)

# Composite fundamental score (equal-weight the available metrics)
fund_cols = ['score_pe', 'score_pfcf', 'score_ev_ebitda', 'score_pb', 'score_peg']
df['fund_score'] = df[fund_cols].mean(axis=1, skipna=True)

# ── Quality sub-scores (higher = better) ────────────────────────────────────
df['score_roe']       = percentile_rank_sector(df['roe'].clip(-0.5, 1),    sector, ascending=False)
df['score_roic']      = percentile_rank_sector(df['roic'].clip(-0.5, 1),   sector, ascending=False)
df['score_op_margin'] = percentile_rank_sector(df['op_margins'].clip(-1,1),sector, ascending=False)
df['score_low_debt']  = percentile_rank_sector(df['debt_equity'].clip(0,10),sector, ascending=True)
df['score_fcf_pos']   = (df['ttm_fcf'] > 0).astype(float)  # Binary: positive FCF = 1

qual_cols = ['score_roe', 'score_roic', 'score_op_margin', 'score_low_debt', 'score_fcf_pos']
df['qual_score'] = df[qual_cols].mean(axis=1, skipna=True)

# ── DCF score ────────────────────────────────────────────────────────────────
# Clip MoS to [-1, 1] so extreme outliers don't dominate
mos_clipped = df['margin_of_safety'].clip(-1.0, 1.0)
# Shift to [0, 1] (0 = fully overvalued, 1 = maximum discount)
df['dcf_score'] = (mos_clipped + 1) / 2
df['dcf_score'] = df['dcf_score'].fillna(0.5)  # neutral for stocks without DCF

# ── Composite ────────────────────────────────────────────────────────────────
df['composite_score'] = (
    W_DCF          * df['dcf_score'] +
    W_FUNDAMENTALS * df['fund_score'] +
    W_QUALITY      * df['qual_score']
)

print('Scoring complete.')
print(f'\nScore distribution:')
print(df[['dcf_score', 'fund_score', 'qual_score', 'composite_score']].describe().round(3))

## 6. Apply Quality Gate & Rank the Universe

In [ ]:
# Minimum quality requirements to be considered
quality_gate = (
    (df['roe'].fillna(0)         >= MIN_ROE) &
    (df['roic'].fillna(0)        >= MIN_ROIC) &
    (df['debt_equity'].fillna(99) <= MAX_DEBT_EQUITY) &
    (df['ttm_fcf'].fillna(-1)    > 0) &
    (df['pe'].fillna(0)          > 0)  # must be profitable
)

df_qualified = df[quality_gate].copy()
print(f'Stocks passing quality gate: {len(df_qualified)} / {len(df)}')

# Final ranking
df_ranked = df_qualified.sort_values('composite_score', ascending=False).reset_index(drop=True)
df_ranked['rank'] = df_ranked.index + 1

# Top 20
top20 = df_ranked.head(20).copy()

display_cols = [
    'rank', 'ticker', 'name', 'sector',
    'price', 'dcf_value', 'margin_of_safety',
    'pe', 'p_fcf', 'ev_ebitda', 'pb', 'peg',
    'roe', 'roic', 'debt_equity',
    'dcf_score', 'fund_score', 'qual_score', 'composite_score'
]

available_cols = [c for c in display_cols if c in top20.columns]
top20_display = top20[available_cols].copy()

# Format for readability
pct_cols = ['margin_of_safety', 'roe', 'roic']
for c in pct_cols:
    if c in top20_display.columns:
        top20_display[c] = top20_display[c].map(lambda x: f'{x:.1%}' if pd.notna(x) else '—')

ratio_cols = ['pe', 'p_fcf', 'ev_ebitda', 'pb', 'peg', 'debt_equity']
for c in ratio_cols:
    if c in top20_display.columns:
        top20_display[c] = top20_display[c].map(lambda x: f'{x:.1f}' if pd.notna(x) else '—')

score_cols = ['dcf_score', 'fund_score', 'qual_score', 'composite_score']
for c in score_cols:
    if c in top20_display.columns:
        top20_display[c] = top20_display[c].map(lambda x: f'{x:.3f}' if pd.notna(x) else '—')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', 30)
display(top20_display)

# Save to CSV
top20.to_csv('output/top20_value_stocks.csv', index=False)
df_ranked.to_csv('output/full_ranking.csv', index=False)
print('\n✓ Results saved to output/')

## 7. Visualisations

In [ ]:
# ── Chart 1: Top 20 Composite Scores ────────────────────────────────────────
fig, ax = plt.subplots(figsize=(12, 7))

colors = plt.cm.YlOrRd(np.linspace(0.4, 0.9, len(top20)))
bars = ax.barh(
    top20['ticker'][::-1],
    top20['composite_score'][::-1],
    color=colors[::-1],
    edgecolor='none',
    height=0.7
)

# Annotate with scores
for bar, (_, row) in zip(bars, top20[::-1].iterrows()):
    ax.text(bar.get_width() + 0.002, bar.get_y() + bar.get_height()/2,
            f'{row["composite_score"]:.3f}',
            va='center', ha='left', fontsize=8, color='#aaa')

ax.set_xlabel('Composite Score (higher = more attractive)', fontsize=11)
ax.set_title('Top 20 S&P 500 Value Stocks — 5-Year Horizon', fontsize=14, pad=15)
ax.spines[['top','right']].set_visible(False)
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('output/top20_scores.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Chart 2: Score Breakdown (stacked bar) ───────────────────────────────────
fig, ax = plt.subplots(figsize=(13, 7))

tickers   = top20['ticker'].tolist()
dcf_vals  = (top20['dcf_score']   * W_DCF).tolist()
fund_vals = (top20['fund_score']  * W_FUNDAMENTALS).tolist()
qual_vals = (top20['qual_score']  * W_QUALITY).tolist()

x = np.arange(len(tickers))
w = 0.65

b1 = ax.bar(x, dcf_vals,  width=w, label=f'DCF ({W_DCF:.0%})',           color='#e67e22')
b2 = ax.bar(x, fund_vals, width=w, bottom=dcf_vals,
            label=f'Fundamentals ({W_FUNDAMENTALS:.0%})',                  color='#3498db')
b3 = ax.bar(x, qual_vals, width=w,
            bottom=[d+f for d,f in zip(dcf_vals, fund_vals)],
            label=f'Quality ({W_QUALITY:.0%})',                            color='#2ecc71')

ax.set_xticks(x)
ax.set_xticklabels(tickers, rotation=45, ha='right', fontsize=9)
ax.set_ylabel('Weighted Score Contribution')
ax.set_title('Score Breakdown by Pillar — Top 20', fontsize=13, pad=12)
ax.legend(loc='upper right', framealpha=0.2)
ax.spines[['top','right']].set_visible(False)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('output/score_breakdown.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Chart 3: Price vs. DCF Intrinsic Value ───────────────────────────────────
plot_df = top20[top20['dcf_value'].notna()].copy()

fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(plot_df))
w = 0.35

ax.bar(x - w/2, plot_df['price'],     width=w, label='Current Price',   color='#3498db', alpha=0.85)
ax.bar(x + w/2, plot_df['dcf_value'], width=w, label='DCF Intrinsic Value', color='#e67e22', alpha=0.85)

ax.set_xticks(x)
ax.set_xticklabels(plot_df['ticker'], rotation=45, ha='right', fontsize=9)
ax.set_ylabel('Price ($)')
ax.set_title('Current Price vs. DCF Intrinsic Value — Top 20', fontsize=13, pad=12)
ax.legend(framealpha=0.2)
ax.spines[['top','right']].set_visible(False)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('output/price_vs_dcf.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Chart 4: Sector Distribution ─────────────────────────────────────────────
sector_counts = top20['sector'].value_counts()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
palette = sns.color_palette('husl', len(sector_counts))
ax1.pie(sector_counts.values, labels=sector_counts.index,
        colors=palette, autopct='%1.0f%%', startangle=90,
        textprops={'fontsize': 9})
ax1.set_title('Top 20 — Sector Breakdown', fontsize=12)

# Scatter: Quality vs. Valuation coloured by DCF MoS
scatter_df = df_qualified.dropna(subset=['fund_score', 'qual_score', 'margin_of_safety'])
sc = ax2.scatter(
    scatter_df['fund_score'],
    scatter_df['qual_score'],
    c=scatter_df['margin_of_safety'].clip(-0.5, 1),
    cmap='RdYlGn',
    alpha=0.5, s=20
)
# Highlight top 20
ax2.scatter(top20['fund_score'], top20['qual_score'],
            color='gold', s=80, zorder=5, label='Top 20', edgecolors='white', linewidth=0.5)
for _, row in top20.iterrows():
    ax2.annotate(row['ticker'], (row['fund_score'], row['qual_score']),
                 fontsize=6.5, color='white', alpha=0.9,
                 xytext=(3, 3), textcoords='offset points')

plt.colorbar(sc, ax=ax2, label='DCF Margin of Safety')
ax2.set_xlabel('Fundamental Score (Valuation)')
ax2.set_ylabel('Quality Score')
ax2.set_title('Quality vs. Valuation (S&P 500 universe)', fontsize=12)
ax2.legend(framealpha=0.2, fontsize=8)
ax2.spines[['top','right']].set_visible(False)

plt.tight_layout()
plt.savefig('output/sector_and_scatter.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Chart 5: Key Valuation Ratios Heatmap ────────────────────────────────────
heatmap_cols = ['pe', 'p_fcf', 'ev_ebitda', 'pb', 'peg']
heatmap_df = top20.set_index('ticker')[heatmap_cols].apply(pd.to_numeric, errors='coerce')

# Normalise each column to [0, 1] for colour (lower = greener = better)
norm_df = heatmap_df.copy()
for col in heatmap_cols:
    mn, mx = norm_df[col].min(), norm_df[col].max()
    if mx > mn:
        norm_df[col] = 1 - (norm_df[col] - mn) / (mx - mn)

fig, ax = plt.subplots(figsize=(10, 9))
sns.heatmap(
    norm_df,
    annot=heatmap_df.round(1),
    fmt='.1f',
    cmap='RdYlGn',
    linewidths=0.5,
    linecolor='#222',
    ax=ax,
    cbar=False,
    annot_kws={'size': 9}
)
ax.set_title('Valuation Ratios — Top 20 Stocks (Green = Cheaper)', fontsize=12, pad=10)
ax.set_xlabel('')
ax.set_xticklabels(['P/E', 'P/FCF', 'EV/EBITDA', 'P/B', 'PEG'], fontsize=10)
plt.tight_layout()
plt.savefig('output/ratios_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Summary Report

In [ ]:
print('=' * 70)
print('  S&P 500 VALUE SCREENER — TOP 20 STOCKS (5-YEAR HORIZON)')
print(f'  Run date: {datetime.today().strftime("%B %d, %Y")}')
print('=' * 70)
print()
print(f'  Universe scanned:  {len(df)} S&P 500 companies')
print(f'  Passed quality gate: {len(df_qualified)}')
print()
print('  Scoring weights:')
print(f'    DCF margin of safety  {W_DCF:.0%}')
print(f'    Fundamental ratios    {W_FUNDAMENTALS:.0%}')
print(f'    Quality (ROE/ROIC)    {W_QUALITY:.0%}')
print()
print('  DCF assumptions:')
print(f'    Discount rate (WACC)  {WACC:.0%}')
print(f'    Terminal growth rate  {TERMINAL_GROWTH:.0%}')
print(f'    Projection horizon    {PROJECTION_YEARS} years')
print(f'    FCF growth cap        {MAX_FCF_GROWTH:.0%} / {MIN_FCF_GROWTH:.0%}')
print()
print('-' * 70)
print(f'  {"#":<4} {"Ticker":<8} {"Company":<28} {"Sector":<22} {"Score"}')
print('-' * 70)
for _, row in top20.iterrows():
    name_trunc = str(row.get('name', ''))[:27]
    sector_trunc = str(row.get('sector', ''))[:21]
    print(f'  {int(row["rank"]):<4} {row["ticker"]:<8} {name_trunc:<28} {sector_trunc:<22} {row["composite_score"]:.4f}')
print('-' * 70)
print()
print('  Files saved:')
print('    output/top20_value_stocks.csv   — Full data for top 20')
print('    output/full_ranking.csv         — All qualified stocks ranked')
print('    output/top20_scores.png         — Bar chart of composite scores')
print('    output/score_breakdown.png      — Pillar contribution breakdown')
print('    output/price_vs_dcf.png         — Price vs. intrinsic value')
print('    output/sector_and_scatter.png   — Sector pie + quality/value scatter')
print('    output/ratios_heatmap.png       — Valuation ratios heatmap')
print()
print('  ⚠  DISCLAIMER: This is a quantitative screening tool, not financial')
print('     advice. Always conduct your own due diligence before investing.')
print('=' * 70)

## 9. Tweak the Model (Optional)

Run the cell below to quickly re-rank with different weights — no re-downloading needed.

In [ ]:
# ── Re-rank with custom weights ──────────────────────────────────────────────
# Change these and re-run this cell:
CUSTOM_W_DCF          = 0.40
CUSTOM_W_FUNDAMENTALS = 0.35
CUSTOM_W_QUALITY      = 0.25

assert abs(CUSTOM_W_DCF + CUSTOM_W_FUNDAMENTALS + CUSTOM_W_QUALITY - 1.0) < 1e-9, \
    'Weights must sum to 1.0'

df_qualified['custom_score'] = (
    CUSTOM_W_DCF          * df_qualified['dcf_score'] +
    CUSTOM_W_FUNDAMENTALS * df_qualified['fund_score'] +
    CUSTOM_W_QUALITY      * df_qualified['qual_score']
)

custom_top20 = df_qualified.sort_values('custom_score', ascending=False).head(20)

print(f'Top 20 with weights DCF={CUSTOM_W_DCF:.0%}  Fund={CUSTOM_W_FUNDAMENTALS:.0%}  Quality={CUSTOM_W_QUALITY:.0%}')
print()
for i, (_, row) in enumerate(custom_top20.iterrows(), 1):
    name_trunc = str(row.get('name', ''))[:28]
    print(f'  {i:<3} {row["ticker"]:<8} {name_trunc:<30} {row["custom_score"]:.4f}')